In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
import joblib

def retrain_model_v2():
    print("🚀 Iniciando Entrenamiento del Modelo v2.0 (Multi-Patología)...")
    
    # 1. Cargar Dataset Aumentado
    try:
        df = pd.read_csv("nhanes_augmented_data.csv")
        print(f"📄 Datos cargados: {len(df)} pacientes.")
    except FileNotFoundError:
        print("❌ Error: No encuentro 'nhanes_augmented_data.csv'.")
        return

    # 2. Preprocesamiento
    # X = Features, y = Diagnosis
    # Asegúrate de eliminar columnas que no son predictores (ID)
    X = df.drop(columns=['Diagnosis', 'ID']) 
    y_raw = df['Diagnosis']
    
    # Codificar las Nuevas Clases
    le = LabelEncoder()
    y = le.fit_transform(y_raw)
    
    # Guardar el encoder v2 (CRÍTICO: Ahora tiene más clases)
    joblib.dump(le, 'label_encoder_v2.joblib')
    print(f"🏷️ Clases detectadas ({len(le.classes_)}): {le.classes_}")

    # 3. Configuración XGBoost v2
    model = xgb.XGBClassifier(
        objective='multi:softprob', 
        num_class=len(le.classes_), 
        eval_metric='mlogloss',
        random_state=42,
        n_estimators=200,    # Aumentamos un poco para aprender más matices
        max_depth=6,         # Un poco más profundo para patrones complejos
        learning_rate=0.05
    )

    # 4. Validación Cruzada
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []
    
    print("\n⚔️  Validando rendimiento (Stratified K-Fold)...")
    
    fold = 1
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)
        print(f"   🔹 Fold {fold}: F1-Weighted={f1:.4f}")
        fold += 1

    print(f"🏆 Promedio F1-Weighted: {np.mean(f1_scores):.4f}")
    
    # 5. Entrenamiento Final (Full Data)
    print("\n📈 Entrenando modelo final v2...")
    model.fit(X, y)
    
    # Guardar modelo v2
    model.save_model("xgboost_clinical_v2.json")
    print("💾 Modelo guardado: xgboost_clinical_v2.json")
    
    # 6. Reporte de Clasificación (Sanity Check)
    # Queremos ver si distingue la 'Sospecha Leucemia'
    print("\n🔍 Desempeño por Patología:")
    final_preds = model.predict(X)
    print(classification_report(y, final_preds, target_names=le.classes_))

if __name__ == "__main__":
    retrain_model_v2()

🚀 Iniciando Entrenamiento del Modelo v2.0 (Multi-Patología)...
📄 Datos cargados: 8177 pacientes.
🏷️ Clases detectadas (7): ['Anemia' 'Anemia Ferropénica' 'Anemia Megaloblástica' 'Infection'
 'Normal' 'Sospecha Leucemia' 'Thrombocytopenia']

⚔️  Validando rendimiento (Stratified K-Fold)...
   🔹 Fold 1: F1-Weighted=0.9994
   🔹 Fold 2: F1-Weighted=1.0000
   🔹 Fold 3: F1-Weighted=1.0000
   🔹 Fold 4: F1-Weighted=0.9957
   🔹 Fold 5: F1-Weighted=1.0000
🏆 Promedio F1-Weighted: 0.9990

📈 Entrenando modelo final v2...
💾 Modelo guardado: xgboost_clinical_v2.json

🔍 Desempeño por Patología:
                       precision    recall  f1-score   support

               Anemia       1.00      1.00      1.00      1060
   Anemia Ferropénica       1.00      1.00      1.00       300
Anemia Megaloblástica       1.00      1.00      1.00       200
            Infection       1.00      1.00      1.00       440
               Normal       1.00      1.00      1.00      5792
    Sospecha Leucemia       1.00   